In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install openai==0.28

In [ ]:
import openai
import os


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# print(openai.__version__)

In [ ]:
!pip install langchain PyPDF2

In [ ]:
! pip install -U langchain-community

In [ ]:
! pip install --upgrade langchain

In [ ]:
!pip install pypdf

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
def get_pdf_text(pdf_docs):
    text = ""
    pdf_reader = PdfReader(pdf_docs)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [ ]:
text = get_pdf_text('/content/drive/MyDrive/RAG/BabarAzam70.pdf')

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=100,
        chunk_overlap=20,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    output_dir = "output_multiple"
    os.makedirs(output_dir, exist_ok=True)
    for i, chunk in enumerate(chunks):
        file_path = os.path.join(output_dir, f"chunk_{i + 1}.json")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(chunk)
    return chunks

In [ ]:
chunks = get_text_chunks(text)

In [ ]:
len(chunks)

52

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
!pip install faiss-cpu

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
!pip install openai==0.28

# VectorStore and embedding

In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.4 MB/s eta 0:00:00


In [ ]:
def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [ ]:
embd = get_vectorstore(chunks)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI(openai_api_key=openai_api_key)
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
def respond_to_query(query, conversation_chain):
    full_response = conversation_chain({"question": query})
    return full_response['answer']

In [ ]:
vectorstore = get_vectorstore(chunks)
conversation_chain = get_conversation_chain(vectorstore)
user_query = input()
response = respond_to_query(user_query, conversation_chain)
print(response)

how many balls he completed his 96 runs ?
It was mentioned in the context that he completed his 96 runs off 98 balls.
